# Create Infrastructure Barcharts

### Imports

In [16]:
import pandas as pd
import plotly.express as px
import kaleido
import numpy as np

import os

### Settings

In [2]:
color_map_dict={
    'Natural Gas':'#bdb59d',
    'Coal': '#75736d',
    'Solar PV': '#F7B615',
    'Solar CSP': '#f1781b',
    'Wind': '#95C060', 
    'Nuclear': '#6D547C', 
    'Offshore Wind':'#2B7F9E', 
    'Biomass': '#713275',
    'Geothermal': '#BB3B1C', 
    'Oil': '#BB3B1C',
    'Hydro': '#14345e'}

### Functions

In [3]:
def plot_infrastructure_bar(df, scenario, title, year, output_file_name):

    df_plot = df[df.scenario==scenario].copy()
    df_plot['unit_size_mw'] = df_plot['unit_size_mw'] / 1000
    df_plot = df_plot.rename(columns={'unit_size_mw': 'GW', 'tech_name_simple':'Technology'})

    # collect new infrastructure
    new_df = df_plot[df_plot.cerf_sited == 1].copy()
    new_df['Year'] = new_df.sited_year
    new_df = new_df.groupby(['Technology', 'Year'], as_index=False).sum()

    # collect new infrastructure
    ret_df = df_plot[df_plot.cerf_sited == 0].copy()
    ret_df['GW'] = ret_df['GW'] * -1
    ret_df = ret_df[ret_df.retirement_year <= year]
    ret_df['Year'] = ret_df.retirement_year
    ret_df = ret_df.groupby(['Technology', 'Year'], as_index=False).sum()

    fig_df = pd.concat([new_df, ret_df])

    fig = px.bar(fig_df, x="Year", y='GW', color="Technology", title=title, color_discrete_map=color_map_dict, width=900, height=500)
    fig.update_yaxes(range=[-75, 150])    
    fig.update_layout(uniformtext_minsize=18, uniformtext_mode='hide',  plot_bgcolor = 'white', paper_bgcolor ='white')
    fig.update_yaxes(title="GW", showgrid = True,  gridcolor = '#bdbdbd')
    fig.add_hline(y=0,line_width=3, line_dash="dash", line_color="black")

    fig.update_xaxes(tickmode = 'array',
                     tickvals = list(range(2025, year+1, 5)),
                     ticktext= ['2025', '2030', '2035', '2040', '2045', '2050'])
    fig.update_traces(width=3)

    fig.write_image(os.path.join(fig_dir , output_file_name))
    
    return fig.show()


def plot_state_bar_capacity(df, scenario, year, output_file_name, title):

    df_bar = df[df.scenario==scenario].copy()
    
    df_bar = df_bar[df_bar.cerf_sited==1]
    
    df_bar = df_bar[['region_name', 'tech_name_simple', 'unit_size_mw']].groupby(['region_name', 'tech_name_simple'], as_index=False).sum()
    
    df_bar = df_bar.rename(columns={'tech_name_simple':'Technology', 'region_name': 'State'})
    
    df_bar['unit_size_mw'] = df_bar['unit_size_mw']/1000
    
    fig = px.bar(df_bar, x="State", y='unit_size_mw', color="Technology", title=title,
                color_discrete_map=color_map_dict,
                width=1000, height=500)
    fig.update_layout(uniformtext_minsize=18, uniformtext_mode='hide', yaxis_range=[0,160], plot_bgcolor = 'white', paper_bgcolor ='white')
    fig.update_yaxes(title="GW", showgrid = True,  gridcolor = '#bdbdbd')

    fig.update_xaxes(categoryorder='category ascending')

    fig.write_image(os.path.join(fig_dir , output_file_name))
    
    return fig.show()


def plot_state_bar_land(df, scenario, year, output_file_name, title):

    df_bar = df[df.scenario==scenario].copy()
    
    df_bar = df_bar[df_bar.cerf_sited==1]
    
    df_bar = df_bar.groupby(['region_name', 'tech_name_simple'], as_index=False).count()
    
    df_bar = df_bar.rename(columns={'tech_name_simple':'Technology', 'region_name': 'State'})
    
    fig = px.bar(df_bar, x="State", y='xcoord', color="Technology", title=title,
                color_discrete_map=color_map_dict,
                width=1000, height=500)
    fig.update_layout(uniformtext_minsize=18, uniformtext_mode='hide', yaxis_range=[0,4500], plot_bgcolor = 'white', paper_bgcolor ='white')
    fig.update_yaxes(title="square km", showgrid = True,  gridcolor = '#bdbdbd')
    
    fig.update_xaxes(categoryorder='category ascending')

    fig.write_image(os.path.join(fig_dir , output_file_name))
    
    return fig.show()

### Data Paths

In [8]:
# data dir
data_dir = os.path.join(os.path.dirname(os.getcwd()), 'data', 'input_data')

# output figure dir
fig_dir = os.path.join(os.path.dirname(os.getcwd()), 'figures', 'infrastructure_barcharts')

# infrastructure siting output csv
infrastucture_path = os.path.join(data_dir, 'infrastructure_data_csv', 'infrastructure_data_2050.csv')

### Collect Data

In [5]:
# read in infrastructure data
df = pd.read_csv(infrastucture_path)

# Create Figures

In [6]:
year = 2050

# reduce to year of interest
df = df[df.sited_year <=year]

### Capacity Additions and Retirements by Technology

#### Clean Grid

In [9]:
plot_infrastructure_bar(df=df, scenario='net_zero_ira_ccs_climate', title='Clean Grid, New Capacity Additions and Retirements by Year', 
                                 year=year, output_file_name=f'clean_grid_barchart_{year}.png')

#### Business-as-usual

In [10]:
plot_infrastructure_bar(df=df, scenario='business_as_usual_ira_ccs_climate', title='Business-as-usual, New Capacity Additions and Retirements by Year', 
                                 year=year, output_file_name=f'bau_barchart_{year}.png')

________________

### Capacity Additions by Technology & State

#### Clean Grid

In [9]:
plot_state_bar_capacity(df=df, scenario='net_zero_ira_ccs_climate', year=year,
         output_file_name = f'clean_grid_{year}_state_tech_capacity.png',
         title = f'Clean Grid, Total Capacity by Technology and State for New Infrastructure by {year}')

#### Business-as-usual

In [10]:
plot_state_bar_capacity(df=df, scenario='business_as_usual_ira_ccs_climate', year=year,
         output_file_name = f'bau_{year}_state_tech_capacity.png',
         title = f'Business-as-Usual, Total Capacity by Technology and State for New Infrastructure by {year}')

________________

## Land Usage by Technology & State

#### Clean Grid

In [11]:
plot_state_bar_land(df=df, scenario='net_zero_ira_ccs_climate', year=year,
         output_file_name = f'clean_grid_{year}_state_tech_land_usage.png',
         title = f'Clean Grid, Total Land Usage by Technology and State for New Infrastructure by {year}')

#### Business-as-usual

In [13]:
plot_state_bar_land(df=df, scenario='business_as_usual_ira_ccs_climate', year=year,
         output_file_name = f'bau_{year}_state_tech_land_usage.png',
         title = f'Business-as-Usual, Land Usage by Technology and State for New Infrastructure by {year}')

### Comparison of Land Usage and Capacity Additions by State and Scenario

In [84]:
df_bar = df.copy()
df_bar = df_bar[df_bar.cerf_sited==1]

df_cap = df_bar.copy()
df_cap = df_cap[['scenario', 'region_name', 'tech_name_simple', 'unit_size_mw']].groupby(['scenario','region_name', 'tech_name_simple'], as_index=False).sum()
df_cap = df_cap.rename(columns={'tech_name_simple':'Technology', 'region_name': 'State'})
df_cap['GW'] = df_cap['unit_size_mw']/1000

df_cap = pd.melt(df_cap, id_vars=['Technology', 'State', 'scenario'], value_vars=['GW'],
        var_name='var', value_name='value')

df_km = df_bar.copy()
df_km = df_km[['scenario', 'region_name', 'tech_name_simple', 'unit_size_mw']].groupby(['scenario','region_name', 'tech_name_simple'], as_index=False).count()
df_km = df_km.rename(columns={'tech_name_simple':'Technology', 'region_name': 'State', 'unit_size_mw':'km-squared'})
df_km = pd.melt(df_km, id_vars=['Technology', 'State', 'scenario'], value_vars=['km-squared'],
        var_name='var', value_name='value')


df_plot = pd.concat([df_cap, df_km])

df_plot['scenario'].replace('business_as_usual_ira_ccs_climate', 'Business-as-usual', inplace=True)
df_plot['scenario'].replace('net_zero_ira_ccs_climate', 'Clean Grid', inplace=True)

In [138]:


fig = px.bar(df_plot, x="State", y='value', color="Technology", title='',
            color_discrete_map=color_map_dict,
            width=900, height=700, 
             facet_col="scenario", facet_row='var',
            facet_col_spacing=0.1,
            facet_row_spacing=0.1)

fig.update_layout(uniformtext_minsize=18, uniformtext_mode='hide', plot_bgcolor = 'white', paper_bgcolor ='white')

fig.update_yaxes(showgrid = True,  gridcolor = '#bdbdbd',showticklabels=True, visible=True)



# makes the y-axes match in each row
fig.layout.yaxis.matches = 'y'
fig.layout.yaxis2.matches = 'y'
fig.layout.yaxis3.matches = 'y4'
fig.layout.yaxis4.matches = 'y4'

# Updates y-axis labels
fig.update_xaxes(title_text="", row=1, col=1)
fig.update_yaxes(title_text="Capacity Additions (GW)", row=2, col=1)
fig.update_xaxes(title_text="", row=1, col=2)
fig.update_yaxes(title_text="Land Usage (km<sup>2</sup>)", row=1, col=1)

# suppresses row facet labels
fig.for_each_annotation(lambda a: a.update(text=a.text.replace("GW", "")))
fig.for_each_annotation(lambda a: a.update(text=a.text.replace("km-squared", "")))
fig.for_each_annotation(lambda a: a.update(text=a.text.replace("Business-as-usual", "")))
fig.for_each_annotation(lambda a: a.update(text=a.text.replace("Clean Grid", "")))
fig.for_each_annotation(lambda a: a.update(text=a.text.split("=")[-1]))

fig.update_layout(
    font=dict(
        family="Arial",
        size=12,  # Set the font size here
        color="black"
    )
)
fig.add_annotation(
    text="Business-as-usual",
    x=0.16, y=1.05,  # Adjust the position
    xref="paper", yref="paper",
    showarrow=False,
    font=dict(size=15),  # Adjust the font size here
)

fig.add_annotation(
    text="Clean Grid",
    x=0.8, y=1.05,  # Adjust the position
    xref="paper", yref="paper",
    showarrow=False,
    font=dict(size=15),  # Adjust the font size here
)

fig.add_annotation(
    text="(a)",
    x=-.05, y=1.1,  # Adjust the position
    xref="paper", yref="paper",
    showarrow=False,
    font=dict(size=18),  # Adjust the font size here
)

fig.add_annotation(
    text="(b)",
    x=.5, y=1.1,  # Adjust the position
    xref="paper", yref="paper",
    showarrow=False,
    font=dict(size=18),  # Adjust the font size here
)

fig.add_annotation(
    text="(c)",
    x=-.05, y=.5,  # Adjust the position
    xref="paper", yref="paper",
    showarrow=False,
    font=dict(size=18),  # Adjust the font size here
)

fig.add_annotation(
    text="(d)",
    x=.5, y=.5,  # Adjust the position
    xref="paper", yref="paper",
    showarrow=False,
    font=dict(size=18),  # Adjust the font size here
)

#fig.write_image(os.path.join(fig_dir , output_file_name))
fig.show()

### Comparison of Land Usage Between Scenarios

In [139]:
df_bar = df.copy()
df_bar = df_bar[df_bar.cerf_sited==1]
    
df_bar = df_bar[['scenario', 'tech_name_simple', 'unit_size_mw']].groupby(['scenario','tech_name_simple'],
                                                                                         as_index=False).count()
    
df_bar = df_bar.rename(columns={'tech_name_simple':'Technology', 'unit_size_mw':'km'})

fig = px.bar(df_bar, y='km', x="scenario", title='',
             color="Technology",
             color_discrete_map=color_map_dict,
                width=800, height=500)

fig.update_layout(uniformtext_minsize=18, uniformtext_mode='hide', yaxis_range=[0,26000], plot_bgcolor = 'white', paper_bgcolor ='white', 
                 )
fig.update_layout(showlegend=True)
fig.update_yaxes(title="square km", showgrid = True,  gridcolor = '#bdbdbd')

fig.update_traces(width=.5)
fig.show()

In [142]:
25002 - 17736

7266

In [141]:
df_bar.groupby('scenario').sum()

,Technology,km
scenario,,
business_as_usual_ira_ccs_climate,BiomassCoalNatural GasOffshore WindSolar CSPSo...,17736
net_zero_ira_ccs_climate,BiomassCoalNatural GasOffshore WindSolar CSPSo...,25002


### Comparison of New Capacity Deployments Between Scenarios

In [38]:
df_bar = df.copy()
df_bar = df_bar[df_bar.cerf_sited==1]
    
df_bar = df_bar[['scenario', 'tech_name_simple', 'unit_size_mw']].groupby(['scenario','tech_name_simple'],
                                                                                         as_index=False).sum()
    
df_bar = df_bar.rename(columns={'tech_name_simple':'Technology', 'unit_size_mw':'MW'})

fig = px.bar(df_bar, y='MW', x="scenario", title='',
             color="Technology",
             color_discrete_map=color_map_dict,
                width=800, height=500)

fig.update_layout(uniformtext_minsize=18, uniformtext_mode='hide', yaxis_range=[0,625000], plot_bgcolor = 'white', paper_bgcolor ='white', 
                 )
fig.update_layout(showlegend=True)
fig.update_yaxes(title="MW", showgrid = True,  gridcolor = '#bdbdbd')

fig.update_traces(width=.5)
fig.show()